<div align="right" style="text-align: right"><i>Peter Norvig<br>Aug 2020</i></div>

# WAR: [What is it Good For?](https://www.youtube.com/watch?v=bX7V6FAoTLc)

The [538 Riddler Classic for 28 August 2020](https://fivethirtyeight.com/features/can-you-cover-the-globe/) asks about the probability of winning the children's [card game **war**](https://en.wikipedia.org/wiki/War_%28card_game%29) in a **sweep**: player **A** wins 26 turns in a row against player **B**. (In **war**, players are dealt 26 cards each and on  each turn they flip the top card in their respective hands; higher rank card wins. Other rules are not relevant to this problem.)

We'll analyze this problem and come up with a program to solve it; first let's get the imports out of the  way:

In [1]:
import random
import collections
from statistics import mean
from itertools  import permutations

In my analysis I considered four different approaches to the problem, which I will describe in the order I considered them (although I actually implemented the final one first, and then came back to fill in the implementation of the earlier approaches).

# Approach 1: Simple Arithmetic?

A naive approach reasons as follows: there are 13 ranks, so perhaps the outcome probabilities for the first turn are:

     A wins=6/13; B wins=6/13; tie=1/13
     
And thus the probability that **A** wins 26 turns in a row would be:

In [2]:
(6/13) ** 26

1.8595392516568175e-09

or about 2 in a billion. Unfortunately, that reasoning is **wrong**, even for one turn. The probability of a tie (that is, both players flipping cards of the same rank) on the first turn is actually 3/51 (not 4/52) because after player **A** flips a card, there are 51 equiprobable cards remaining for player **B**, and 3 have the  same rank. So we actually have for the first turn:

    A wins=24/51; B wins=24/51; tie=3/51

The probabilities on subsequent turns would be slightly different, depending on the cards picked in the previous turns. So simple arithmetic doesn't give us the answer.

# Approach 2: Brute Force Enumeration?

Brute force enumeration means:
- Consider every possible permutation of the deck of cards.
- For each permutation, deal out the cards and see whether or not **A** sweeps.
- The probability that **A** sweeps is the number of sweeps divided by the number of permutations.

Easy-peasy; here's the code. First the types:

In [3]:
Probability = float # Type: Probability is a number between 0.0 and 1.0
Deck = list         # Type: Deck is a list of card ranks

Now the functions:

In [4]:
def make_deck(ranks=13, suits=4) -> Deck: 
    """Make a list of ranks, each rank repeated `suits` times."""
    return list(range(ranks)) * suits

def A_sweeps(deck) -> bool: 
    """Upon dealing this deck, does player A win every turn?"""
    return all(deck[i] > deck[i + 1] for i in range(0, len(deck), 2))

def brute_p_sweep(deck) -> Probability:
    """The probability that A sweeps, considering every permutation of deck."""
    return mean(A_sweeps(d) for d in permutations(deck))

(Note that in Python the `bool` value `False` is equivalent to `0` and `True` is equivalent to `1`, so the `mean` of an iterable of `bool` values is the same as the proportion or probability of `True` values.)

Here we run the code on a tiny deck of just 8 cards, all with different ranks:

In [5]:
make_deck(8, 1)

[0, 1, 2, 3, 4, 5, 6, 7]

In [6]:
brute_p_sweep(make_deck(8, 1))

0.0625

There can be no ties in this deck, so this is just four turns where on each turn **A** and **B** each have an equal chance of winning, so the probability of **A** winning all 4 turns is: 

In [7]:
(1/2) ** 4

0.0625

If the 8-card deck had 4 ranks and 2 suits, then the probability of **A** sweeping would be less, because there could be ties:

In [8]:
make_deck(4, 2)

[0, 1, 2, 3, 0, 1, 2, 3]

In [9]:
brute_p_sweep(make_deck(4, 2))

0.03571428571428571

What about the real deck, with 52 cards? Unfortunately, there are 52! permutations (more than $10^{67}$), and even if we were clever about the duplicated ranks and the ordering of the 26 turns, and
even if we could process a billion deals a second, it would still take [millions of years](https://www.google.com/search?q=%2852%21+%2F+4%21%5E13+%2F+26%21%29+nanoseconds+in+years&oq=%2852%21+%2F+4%21%5E13+%2F+26%21%29+nanoseconds+in+years) to complete the brute force enumeration. And 538 wanted the answer by Monday.



# Approach 3: Simulation?

It is simple to code up a simulation that **randomly samples** the space of possible permutations rather than processing **all** the permutations. Just repeatedly shuffle the cards and compute how often **A** sweeps:

In [10]:
def deals(deck, N) -> Deck: 
    """Yield N randomly shuffled deals of deck."""
    for _ in range(N):
        random.shuffle(deck)
        yield deck

def simulate(deck, N=10000) -> Probability:
    """Simulate N games of war, and return the estimated probability of A sweeping."""
    return mean(A_sweeps(d) for d in deals(deck, N))

In [11]:
simulate(make_deck())

0

Examining 10,000 deals with the full 52-card deck we got zero sweeps. Since the probability of a sweep is probably somewhere around one in a billion, we  would need to look at trillions of deals to get a reliable estimate. That would require over a day of run time: much better than millions of years, but still not good enough.

# Approach 4: Abstract Incremental Enumeration!

The first three approaches didn't pan out. What next? 

It is feasible to consider every possible deal if we are careful. As discussed in my [How To Count Things](How%20To%20Count%20Things.ipynb) notebook, the idea is to represent a deck not as a concrete permutation of 52 ranks but rather with a representation that is:
- **Abstract**: What really matters is not the exact identity of every card in the deck, but rather whether **A**'s next card is higher, lower, or the same as **B**'s next card. For example, if there are only two cards remaining and we know they have different ranks, then the probability of **A** winning is 1/2; it doesn't matter whether the cards are [10, 8] or [2, 5] or any other combination of two distinct ranks.
- **Incremental**: First we'll consider the possibilities for the two cards in the first turn, and only if **A** wins will we then move on to consider possible cards for the next turn. If **A**  loses or ties the first turn, there is no need to consider the 50! permutations of the remaining cards.


The key is to focus on how many cards in the deck have other cards of the same rank. Thus, we can represent any set of remaining cards with four numbers:
- The number of ranks that have no other card of the same rank.
- The number of ranks that have exactly 2 cards of the same rank.
- The number of ranks that have exactly 3 cards of the same rank.
- The number of ranks that have exactly 4 cards of the same rank. 

An abstract deck (or `ADeck`) is a tuple where `deck[i]` gives the number of different ranks that have `i` cards in the deck. (This representation is a bit wasteful, because `deck[0]` is always a redundant `0`, but the alternative would be to sprinkle the code with a bunch of `[i - 1]` indexes, risking an off-by-one error).

Consider these sample abstract decks:

In [12]:
ADeck = tuple # Type: Abstract Deck, e.g. (0, 0, 0, 0, 13)

deck = (0, 0, 0, 0, 13)
tie1 = (0, 0, 1, 0, 12)
dif1 = (0, 0, 0, 2, 11)
end4 = (0, 4, 0, 0,  0)

- `deck` is the normal 52 card full deck with 13 ranks, each with 4 cards of that rank, so `deck[4]` is 13. (This abstract deck represents 52! concrete decks.)
- `tie1` is the deck that results when both players flip a card of the same rank on the first turn. There are 12 ranks with 4 cards each and 1 rank with 2 cards.
- `dif1` is the deck that results when the players flip cards of different ranks on the first turn.  There  are 11 ranks each with 4 cards and 2 ranks with 3 cards. `tie1` and `dif1` are the only two possible results for turn 1.
- `end4` is a deck you would see near the end of a game, with just 4 remaining cards, each of a different rank. (This represents 52 × 48 × 44 × 40 = 4,392,960 concrete decks.)

We can compute the number of cards in abstract decks as follows:

In [13]:
def ncards(deck: ADeck) -> int:
    """Number of cards remaining in an abstract deck."""
    return sum(i * deck[i] for i in range(len(deck)))

In [14]:
[ncards(d) for d in (deck, tie1, dif1, end4)]

[52, 50, 50, 4]

# Abstract Incremental Enumeration Strategy

Now we're ready to outline a strategy to exactly and efficiently compute `p_sweep(deck)`, the probability that **A** sweeps a game of war:

- Start with `dist` being a **probability distribution** of deck outcomes after 0 turns: `{deck: 1.0}`.
- **for** each of the 26 turns (or in general ncards(deck) / 2 turns):
  - **for** each of the entries in `dist`:
    - See  what possible outcomes can arise from the deck
    - update `dist` to include outcomes where **A** might sweep, with appropriate probabilities 
- Sum the probabilities in `dist`

In [15]:
class PDist(collections.Counter): 
    """A probability distribution of {deck: probability}."""

def p_sweep(deck: ADeck) -> Probability:
    """The probability that player A sweeps a game of war."""
    dist = PDist({deck: 1.0})
    for turn in range(ncards(deck) // 2):
        dist = play_turn(dist)
    return sum(dist.values())

def play_turn(dist) -> PDist:
    """Play one turn with all possible card choices for players A and B; return
    the probability distribution of outcomes where A still might sweep."""
    P = PDist()
    for deck in dist:
        for deck2, p_Awin in select2(deck):
            P[deck2] += dist[deck] * p_Awin
    return P

Now the key is figuring out:
- All the possible ways to select two cards from the deck.
- The probability of that selection.
- Who won (or was it a tie) with that selection.
figuring out what the probab ility of that selection is, figuring out

In [16]:
def select2(deck):
    """All ways to select two cards from deck on a turn, yielding tuples:
    (remaining deck, probability that deck occurs and player A wins)."""
    for deck1, p_i, i in select1(deck):
        for deck2, p_j, j in select1(deck1):
            p_tie  = 0 if j != i - 1 else 1 / deck1[j]
            p_Awin = (1 - p_tie) / 2
            if p_Awin > 0:
                yield deck2, p_i * p_j * p_Awin
            
def select1(deck):
    """All ways to pick one card from deck, returning a list of:
    (remaining deck, probability of pick, index of pick in deck)"""
    Ncards = sum(i * deck[i] for  i in range(len(deck)))
    return [(remove(i, deck), i * deck[i] / Ncards, i)
           for i in range(len(deck)) if deck[i]]

def remove(i, deck):
    """Remove one card from deck[i]."""
    deck1 = list(deck)
    deck1[i] -= 1
    if i - 1 != 0:
        deck1[i - 1] += 1
    return tuple(deck1)

# The Answer!

In [17]:
p_sweep(deck)

3.132436174322294e-09

The probability that **A** sweeps a game of war is a little over 3 in a billion. (The computation took well under a second.)  

# Gaining Confidence in the Answer

Above we computed that the probability of **A** winning the first turn is exactly 24/51. If every other turn were the same, the probability of a sweep would be:

In [18]:
(24/51) ** 26

3.0808297965386556e-09

This is within 2% of the answer we got, giving some credence to our answer.

The brute force algorithm can't handle a big deck, but we can use it to verify that `p_sweep` gets the same answers as `brute_p_sweep` for small decks:

In [19]:
# 6 cards (3 ranks and 2 suits)
{brute_p_sweep(make_deck(3, 2)), p_sweep((0, 0, 3))}

{0.06666666666666667, 0.06666666666666668}

In [20]:
# 6 cards (2 ranks and 3 suits)
{brute_p_sweep(make_deck(2, 3)), p_sweep((0, 0, 0, 2))}

{0.049999999999999996, 0.05}

In [21]:
# 8 cards (8 different ranks)
{brute_p_sweep(make_deck(8, 1)), p_sweep((0, 8))}

{0.0625}

In [22]:
# 8 cards (4 ranks and 2 suits)
{brute_p_sweep(make_deck(4, 2)), p_sweep((0, 0, 4))}

{0.03571428571428571}

In [23]:
# 8 cards (2 ranks and 4 suits)
{brute_p_sweep(make_deck(2, 4)), p_sweep((0, 0, 0, 0, 2))}

{0.014285714285714284, 0.014285714285714285}

We see that in every case the resulting set either has one  member (indicating that the two computations got exactly the same result) or two members that differ only in the final digit (indicating floating-point round-off error).

We can also test `p_sweep` on larger decks that we know the answer for. If there are `2N` cards in a deck, all with different ranks, then the probability that **A** sweeps is `(1/2) ** N`. We can test that:

In [24]:
all(p_sweep((0, 2 * N)) == (1/2) ** N
    for N in range(200))

True

# Working through the algorithm

Let's work through how `p_sweep`, `play_turn`, `select1` and `select2` work. We'll start by reminding ourself what the starting `deck` is, and then `select1` card from it:

In [25]:
deck

(0, 0, 0, 0, 13)

In [26]:
select1(deck)

[((0, 0, 0, 1, 12), 1.0, 4)]

This is saying with probability 1.0 the result of selecting one card is a deck where there are 12 ranks with four-of-a-kind and 1 rank with three-of-a-kind. The selected card came from index 4 in the deck.

Now we'll try selecting two cards:

In [27]:
list(select2(deck))

[((0, 0, 0, 2, 11), 0.47058823529411764)]

This says that the only result of the first turn in which **A** wins has 11 ranks with four-of-a-kind and 2 ranks with 3-of-a-kind. The probability of this outcome is  0.47058823529411764, which, as we computed earlier, is 24/51. The rest of the probability goes to an equiprobable result in which **B** wins, and to `(0, 0, 1, 0, 12)`, which indicates a tie on the first turn. Since these outcomes don't result in **A** winning, they do not appeear in the result from `select2`.

Now let's work through how `p_sweep` repeatedly calls `play_turn`. We start with a probability distribution where we have the initial deck with probability 1.0:

In [28]:
dist = PDist({deck: 1.0})
dist

PDist({(0, 0, 0, 0, 13): 1.0})

Here's the outcome of playing the first turn:

In [29]:
dist = play_turn(dist)
dist

PDist({(0, 0, 0, 2, 11): 0.47058823529411764})

Now for the second turn:

In [30]:
dist = play_turn(dist)
dist

PDist({(0, 0, 2, 0, 11): 0.0017286914765906362,
       (0, 0, 1, 2, 10): 0.05070828331332534,
       (0, 0, 0, 4, 9): 0.16902761104441777})

And the third turn:

In [31]:
dist = play_turn(dist)
dist

PDist({(0, 2, 0, 0, 11): 3.0650558095578654e-06,
       (0, 1, 1, 1, 10): 0.00040458736686163827,
       (0, 0, 2, 2, 9): 0.010114684171540957,
       (0, 1, 0, 3, 9): 0.0017981660749406148,
       (0, 0, 3, 0, 10): 0.00020229368343081916,
       (0, 0, 1, 4, 8): 0.048550484023396595,
       (0, 0, 0, 6, 7): 0.04315598579857475})

We'll leave it as an exercise for the reader to work through these, but one thing you can clearly see is that the total probability of winning is becoming smaller with every turn:

In [32]:
sum(dist.values())

0.10422926617455494

You have only a 10% chance of winning three turns in a row. This will steadily fall to 3-in-a-billion after 26 turns. I have to say, I'm begining to doubt Duane’s friend’s granddaughter's claim that she won 26 in a row.